In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score

# Load dataset
df = pd.read_csv("hopes2.csv")
df.fillna(0, inplace=True)

# Separate features and targets
X = df.drop(columns=['Location', 'Pit', 'PitCompound'])
y_pit = df['Pit']
y_compound = df['PitCompound']

# Encode the PitCompound target variable
label_encoder = LabelEncoder()
y_compound_encoded = label_encoder.fit_transform(y_compound)

# Define the column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Compound']),
        ('scale', StandardScaler(), ['LapTime', 'LapNumber', 'Stint', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'TyreLife', 'TrackStatus', 'Position'])
    ],
    remainder='passthrough'
)

# Function to build and evaluate the model
def build_and_evaluate_model(X, y, model):
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Fit the model
    pipeline.fit(X_train, y_train)

    # Predictions and evaluation
    y_pred = pipeline.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')
    acc = accuracy_score(y_test, y_pred)

    print("F1 Score:", f1)
    print("Accuracy Score:", acc)

    return pipeline

# Best parameters for each model

# XGBoost for Pit
xgb_model_pit = xgb.XGBClassifier(
    objective='binary:logistic',
    random_state=42,

    colsample_bytree=0.9,
    learning_rate=0.2,
    max_depth=5,
    n_estimators=300,
    scale_pos_weight=20,
    subsample=0.9
)

# Random Forest for Pit
rf_model_pit = RandomForestClassifier(
    random_state=42,
    max_depth=20,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100
)

# Random Forest for PitCompound
rf_model_compound = RandomForestClassifier(
    random_state=42,
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100
)

# XGBoost for PitCompound
xgb_model_compound = xgb.XGBClassifier(
    objective='multi:softmax',
    random_state=42,

    colsample_bytree=0.8,
    learning_rate=0.2,
    max_depth=5,
    n_estimators=200,
    scale_pos_weight=1,
    subsample=1.0
)

# Evaluate models
print("Evaluating XGBoost for Pit:")
xgb_model_pit_pipeline = build_and_evaluate_model(X, y_pit, xgb_model_pit)

print("Evaluating Random Forest for Pit:")
rf_model_pit_pipeline = build_and_evaluate_model(X, y_pit, rf_model_pit)

print("Evaluating Random Forest for PitCompound:")
rf_model_compound_pipeline = build_and_evaluate_model(X, y_compound_encoded, rf_model_compound)

print("Evaluating XGBoost for PitCompound:")
xgb_model_compound_pipeline = build_and_evaluate_model(X, y_compound_encoded, xgb_model_compound)

Evaluating XGBoost for Pit:
F1 Score: 0.975903805642832
Accuracy Score: 0.9779231398201145
Evaluating Random Forest for Pit:
F1 Score: 0.9685479343998086
Accuracy Score: 0.9754701553556827
Evaluating Random Forest for PitCompound:
F1 Score: 0.9779594622393757
Accuracy Score: 0.9779231398201145
Evaluating XGBoost for PitCompound:


c:\Users\kedar\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:158: UserWarning: [10:06:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score: 0.9755020975601826
Accuracy Score: 0.9754701553556827


In [2]:
import time

# Load the new race data
new_race_df = pd.read_csv("Nordschielfe.csv")
new_race_df.fillna(0, inplace=True)

# Preprocess the data by dropping the target columns
X_new_race = new_race_df.drop(columns=['Location', 'Pit', 'PitCompound'])

# Iterate through each row in the new race data
for i in range(X_new_race.shape[0]):
    # Extract one row of data
    single_row = X_new_race.iloc[[i]]
    
    # Predict Pit using the trained Random Forest and XGBoost models
    rf_pit_pred = rf_model_pit_pipeline.predict(single_row)
    xgb_pit_pred = xgb_model_pit_pipeline.predict(single_row)
    
    # Predict PitCompound using the trained Random Forest and XGBoost models
    rf_compound_pred = rf_model_compound_pipeline.predict(single_row)
    xgb_compound_pred = xgb_model_compound_pipeline.predict(single_row)
    
    # Decode the PitCompound prediction back to the original labels
    rf_compound_pred_decoded = label_encoder.inverse_transform(rf_compound_pred)
    xgb_compound_pred_decoded = label_encoder.inverse_transform(xgb_compound_pred)
    
    # Print the input row and the predictions
    print(f"Row {i+1}:")
    print(single_row)
    print(f"Predicted Pit (RF): {rf_pit_pred[0]}")
    print(f"Predicted Pit (XGB): {xgb_pit_pred[0]}")
    print(f"Predicted PitCompound (RF): {rf_compound_pred_decoded[0]}")
    print(f"Predicted PitCompound (XGB): {xgb_compound_pred_decoded[0]}")
    print("-" * 50)
    
    # Add a 5-second delay
    time.sleep(5)

Row 1:
   LapTime  LapNumber  Stint  Sector1Time  Sector2Time  Sector3Time  SpeedI1  \
0     95.8          1      1         30.3         39.3         23.8      269   

   SpeedI2  SpeedFL  SpeedST Compound  TyreLife  TrackStatus  Position  
0      224      249      295     SOFT         4            1         3  
Predicted Pit (RF): 0
Predicted Pit (XGB): 0
Predicted PitCompound (RF): SOFT
Predicted PitCompound (XGB): SOFT
--------------------------------------------------
Row 2:
   LapTime  LapNumber  Stint  Sector1Time  Sector2Time  Sector3Time  SpeedI1  \
1     92.6          2      1         30.3         38.6         23.8      264   

   SpeedI2  SpeedFL  SpeedST Compound  TyreLife  TrackStatus  Position  
1      224      248      290     SOFT         5            1         3  
Predicted Pit (RF): 0
Predicted Pit (XGB): 0
Predicted PitCompound (RF): SOFT
Predicted PitCompound (XGB): SOFT
--------------------------------------------------
Row 3:
   LapTime  LapNumber  Stint  Sector1Ti

KeyboardInterrupt: 